In [1]:
# ================================================================
# 🧮 ML_Sales_Prediction.ipynb
# Predict daily sales by product category using Fact_Orders
# Author: Muhammad Saad
# ================================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum as _sum, current_timestamp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


StatementMeta(, ea7c67bc-6171-4989-a80e-e7ba42e3aed1, 3, Finished, Available, Finished)

In [2]:
# 2️⃣ Load Gold Fact_Orders table
fact_orders = spark.read.table("GoldLakehouse.dbo.Fact_Orders")
print(f"Loaded {fact_orders.count()} records from Fact_Orders")


StatementMeta(, ea7c67bc-6171-4989-a80e-e7ba42e3aed1, 4, Finished, Available, Finished)

Loaded 1000 records from Fact_Orders


In [3]:

# 3️⃣ Prepare training dataset: aggregate daily sales per category
sales_df = (
    fact_orders
    .withColumn("Date", to_date(col("Order_Date")))
    .groupBy("Date", "Product_Category")
    .agg(_sum("Order_Total").alias("Daily_Sales"))
    .orderBy("Date")
)

# 4️⃣ Convert to Pandas for ML training
pdf = sales_df.toPandas()
pdf["Date"] = pd.to_datetime(pdf["Date"])
pdf["Day"] = pdf["Date"].dt.day
pdf["Month"] = pdf["Date"].dt.month

# One-hot encode product categories
pdf = pd.get_dummies(pdf, columns=["Product_Category"], drop_first=True)

# 5️⃣ Split data
X = pdf.drop(["Daily_Sales", "Date"], axis=1)
y = pdf["Daily_Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6️⃣ Train ML model
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 7️⃣ Evaluate
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("✅ Model trained successfully")
print(f"📊 MAE: {mae:.2f} | R²: {r2:.3f}")


StatementMeta(, ea7c67bc-6171-4989-a80e-e7ba42e3aed1, 5, Finished, Available, Finished)

✅ Model trained successfully
📊 MAE: 46593.08 | R²: -0.265


In [6]:

# 8️⃣ Make full predictions for all data
pdf["Predicted_Sales"] = model.predict(X)

# 9️⃣ Convert back to Spark DataFrame
predictions_sdf = spark.createDataFrame(pdf[["Date", "Predicted_Sales"] + [c for c in pdf.columns if c.startswith("Product_Category_")]])

predictions_sdf.write.format("delta").mode("overwrite").saveAsTable("Gold_Sales_Predictions")

print("✅ Gold_Sales_Predictions table created successfully")



StatementMeta(, ea7c67bc-6171-4989-a80e-e7ba42e3aed1, 8, Finished, Available, Finished)

✅ Gold_Sales_Predictions table created successfully
